# 📊 TensorFlow 归一化方法对比表
| 方法              | 数学公式                                                     | TensorFlow API                                                                                                                      | 主要参数                     | 输出范围 / 特性                                  | 适用场景                                   |                |                           |
| --------------- | -------------------------------------------------------- | ----------------------------------------------------------------------------------------------------------------------------------- | ------------------------ | ------------------------------------------ | -------------------------------------- | -------------- | ------------------------- |
| **Min-Max 归一化** | $x' = \frac{x - x_{min}}{x_{max} - x_{min}}$             | `tf.keras.layers.Normalization(axis=..., mode="min_max")`  或 手动计算： `(x - tf.reduce_min(x)) / (tf.reduce_max(x) - tf.reduce_min(x))` | `axis`（沿哪个维度计算 min、max）  | $[0,1]$ 或 $[a,b]$（可线性映射）                   | 特征范围差异较大，但分布未知时；常用于图像像素缩放到 \[0,1]      |                |                           |
| **Z-Score 标准化** | $x' = \frac{x - \mu}{\sigma}$                            | `tf.keras.layers.Normalization(axis=..., mean=..., variance=...)` 或 `tf.nn.batch_normalization`                                     | `mean`、`variance`、`axis` | 均值 0，方差 1，数据可能为负                           | 常用于需要均值为 0、方差为 1 的模型（如神经网络输入，梯度下降收敛更快） |                |                           |
| **L1 归一化**      | ( x' = \frac{x}{\|x\|\_1} = \frac{x}{\sum\_i             | x\_i                                                                                                                                | } )                      | `tf.math.l2_normalize(x, ord=1, axis=...)` | `ord=1`、`axis`                         | 使向量的 L1 范数等于 1 | 稀疏特征向量（如 NLP 词袋模型），减少维度影响 |
| **L2 归一化**      | $x' = \frac{x}{\|x\|_2} = \frac{x}{\sqrt{\sum_i x_i^2}}$ | `tf.math.l2_normalize(x, axis=...)`                                                                                                 | `axis`                   | 使向量的 L2 范数等于 1                             | 常用于深度学习特征输入、余弦相似度计算、正则化约束              |                |                           |


## 🔑 关键点说明

- 1.`min-max` → 适合需要固定区间的情况（如图像输入网络前缩放到 [0,1]）。

- 2.`z-score` → 适合机器学习中大多数输入特征的标准化。

- 3.`L1` 归一化 → 常用于稀疏表示，使所有特征权重的绝对和为 `1`。

- 4.`L2` 归一化 → 最常见的向量归一化方法，适合余弦相似度、神经网络输入特征。

# TensorFlow 中的归一化与标准化方法详解

在机器学习和深度学习中，对数据进行适当的预处理至关重要。归一化（Normalization）和标准化（Standardization）是两种常见的预处理技术，它们可以加速模型收敛、提高模型性能并防止数值不稳定。本文介绍 TensorFlow 中常用的四种方法：L2 归一化、Min-Max 归一化、Z-score 标准化和 L1 归一化。

---
# tf.nn.l2_normalize
## L2 归一化 (L2 Normalization)

**定义**：  
`L2 归一化`（也称为欧几里得归一化）将向量的 `L2` 范数（欧几里得长度）缩放到 `1`。即，将向量转换为单位向量。

**特点**：
*   结果向量的 L2 范数等于 1。
*   常用于计算余弦相似度、词向量处理、特征向量归一化等。
*   对异常值（outliers）相对敏感。

In [ ]:
tf.nn.l2_normalize(
    x,              # 输入张量
    axis=None,      # 归一化的维度
    epsilon=1e-12,  # 防止除 0 的小常数
    name=None       # 操作名称
)


| 参数名   | 类型             | 默认值  | 作用说明                                                                 | 备注                     |
|----------|------------------|---------|--------------------------------------------------------------------------|--------------------------|
| x        | Tensor           | 无      | 需要归一化的输入张量，可以是任意维度                                     | 必需参数                 |
| axis     | int 或 list[int] | 无      | 指定归一化维度：<br>- axis=0：按行（跨行）归一化<br>- axis=1：按列（跨列）归一化<br>- 支持多轴如 axis=[1,2] | 必需参数                 |
| epsilon  | float            | 1e-12   | 防止分母为零的极小值                                                     | 可选参数                 |
| name     | string           | 无      | 操作名称                                                                 | 通常无需设置，为可选参数 |

---

# 🔹 L1 归一化
在 `TensorFlow` 中，`L1` 归一化 是通过 `tf.math.l2_normalize` 的 `p=1` 来实现的。

不过官方并没有单独的 `l1_normalize` API，一般我们使用 `tf.norm` + 张量除法的方式。

---
## ✨ 使用方法 1：手动实现

In [ ]:
import tensorflow as tf

# 创建一个张量
x = tf.constant([[1.0, -2.0, 3.0],
                 [4.0, 5.0, -6.0]], dtype=tf.float32)
# # ord:2代表范数，1代表L1范数，fro计算 Frobenius 范数（矩阵的 L2 范数），适用于矩阵
# L1 归一化：每个向量除以其 L1 范数（绝对值和）
# tf.norm()用于计算张量（tensor）的范数（norm）的函数
x_l1 = x / tf.norm(x, ord=1, axis=1, keepdims=True) 
print("原始张量:\n", x.numpy())
print("L1 归一化结果:\n", x_l1.numpy())

输出：
原始张量:
 [[ 1. -2.  3.uu
  [ 4.  5. -6.]]
L1 归一化结果:
 [[ 0.16666667 -0.33333334  0.5]
  [ 0.26666668  0.33333334 -0.4]]


## ✨ 使用方法 2：封装函数

In [ ]:
def l1_normalize(x, axis=None, epsilon=1e-12):
    return x / (tf.reduce_sum(tf.abs(x), axis=axis, keepdims=True) + epsilon)

x_l1 = l1_normalize(x, axis=1)

| 参数/概念        | 说明                                                                        |
| ------------ | ------------------------------------------------------------------------- |
| **x**        | 输入张量，可以是任意维度。                                                             |
| **ord=1**    | 在 `tf.norm` 中指定 L1 范数（绝对值和）。                                              |
| **axis**     | 归一化的维度。<br>- `None`：对整个张量求 L1 范数。<br>- `0`：按列（跨行）归一化。<br>- `1`：按行（跨列）归一化。 |
| **keepdims** | 是否保持维度。`True` 会保留归一化维度，便于广播。                                              |
| **epsilon**  | 避免除零的微小数值，通常设为 `1e-12`。                                                   |
| **返回值**      | 与输入 `x` 形状相同的归一化张量。                                                       |


---

# 🔹 Min-Max 归一化
### 1️⃣ 使用方法

TensorFlow `没有`直接的 `tf.min_max_normalize()` 方法，通常需要手动实现：

In [ ]:
import tensorflow as tf

# 生成随机张量
x = tf.random.uniform(shape=(5,), minval=10, maxval=100, dtype=tf.float32)

# min-max归一化到 [0, 1]
x_min = tf.reduce_min(x)
x_max = tf.reduce_max(x)
x_norm = (x - x_min) / (x_max - x_min)

print("原始张量:", x.numpy())
print("归一化张量:", x_norm.numpy())

# 如果想归一化到 [-1, 1]
x_norm_scaled = x_norm * 2 - 1
print("归一化到 [-1,1]:", x_norm_scaled.numpy())


### 2️⃣ 参数详解表格
| 参数/变量     | 说明                                | 默认值  | 是否必须 |
| --------- | --------------------------------- | ---- | ---- |
| `x`       | 输入张量                              | 无    | ✅    |
| `x_min`   | 输入张量的最小值（用 `tf.reduce_min(x)` 计算） | 动态计算 | ✅    |
| `x_max`   | 输入张量的最大值（用 `tf.reduce_max(x)` 计算） | 动态计算 | ✅    |
| `min_val` | 归一化后目标范围的下界                       | 0    | ❌    |
| `max_val` | 归一化后目标范围的上界                       | 1    | ❌    |

---
### 3️⃣ 归一化到任意区间的实现

In [ ]:
def min_max_normalize(x, min_val=0.0, max_val=1.0):
    x_min = tf.reduce_min(x)
    x_max = tf.reduce_max(x)
    x_norm = (x - x_min) / (x_max - x_min)  # 归一化到 [0,1]
    return x_norm * (max_val - min_val) + min_val

# 示例：归一化到 [5, 10]
x = tf.constant([2.0, 4.0, 6.0, 8.0])
x_scaled = min_max_normalize(x, min_val=5, max_val=10)
print(x_scaled.numpy())


### 👉 所以，Min-Max 归一化
在 TensorFlow 中主要依赖 `tf.reduce_min` 和 `tf.reduce_max` 来动态计算范围，然后手动缩放。

---

# Z-Score 标准化
## 📌 Z-Score 标准化公式:
   - 𝑥′=(𝑥−𝜇)/𝜎

- μ：均值
- σ：标准差

Z-Score 标准化的结果是让数据分布 均值为 0，标准差为 1，常用于机器学习的特征预处理。

## ✅ 在 TensorFlow 中的实现方法
### 方法 1：手动计算

In [ ]:
import tensorflow as tf

# 生成随机张量
x = tf.random.uniform(shape=(5,), minval=0, maxval=10, dtype=tf.float32)

# 计算均值和标准差
mean, variance = tf.nn.moments(x, axes=[0]) # --> 返回 均值 和 方差
std = tf.sqrt(variance)

# Z-Score 标准化
zscore = (x - mean) / std

print("原始数据:", x.numpy())
print("均值:", mean.numpy())
print("标准差:", std.numpy())
print("Z-Score 标准化:", zscore.numpy())


### 方法 2：使用 `tf.nn.batch_normalization`

In [ ]:
import tensorflow as tf

x = tf.random.uniform(shape=(5,), minval=0, maxval=10, dtype=tf.float32)

mean, variance = tf.nn.moments(x, axes=[0])
epsilon = 1e-8  # 防止除 0

zscore = tf.nn.batch_normalization(
    x,
    mean=mean,
    variance=variance,
    offset=None,
    scale=None,
    variance_epsilon=epsilon
)

print("Z-Score 标准化:", zscore.numpy())


- 这种方式其实是 批归一化（Batch Normalization） 的核心步骤之一，可以等价实现 Z-Score 标准化。

#### 📋 参数表格说明（以 tf.nn.batch_normalization 为例）
| 参数                 | 类型                 | 说明                             |
| ------------------ | ------------------ | ------------------------------ |
| `x`                | `Tensor`           | 输入张量（需要标准化的数据）                 |
| `mean`             | `Tensor` (同 dtype) | 数据的均值（一般通过 `tf.nn.moments` 计算） |
| `variance`         | `Tensor` (同 dtype) | 数据的方差（同样通过 `tf.nn.moments` 计算） |
| `offset`           | `Tensor` 或 `None`  | 偏移值，通常设为 `None`（等价于 0）         |
| `scale`            | `Tensor` 或 `None`  | 缩放因子，通常设为 `None`（等价于 1）        |
| `variance_epsilon` | `float`            | 防止除 0 的极小数，默认 `1e-8`           |



## 🎯 总结

- `L1`/`L2` 归一化：让向量的范数为 1。

- `Min`-`Max` 归一化：映射到 [0, 1] 区间。

- `Z-Score` 标准化：调整到均值为 0，方差为 1。

在 TensorFlow 中，Z-Score 标准化常用 手动公式 或 `tf.nn.batch_normalization` 来实现。